In [1]:
import mysql.connector
import configparser

In [17]:
# Leer el archivo de configuración
config = configparser.ConfigParser()
config.read('config.ini')

# Obtener los valores del archivo de configuración
host = config['mysql']['host']
user = config['mysql']['user']
password = config['mysql']['password']

conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password
)

cursor = conn.cursor()

cursor.execute("DROP DATABASE IF EXISTS red_electrica")

cursor.execute("CREATE DATABASE IF NOT EXISTS red_electrica")

conn.commit()

conn.close()

print("Base de datos red_electrica eliminada con éxito si existía.")

Base de datos red_electrica eliminada con éxito si existía.


In [13]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def obtener_fecha_30_dias_atras():
    hoy = datetime.now()
    hace_30_dias = hoy - timedelta(days=1) 
    return hace_30_dias.strftime("%Y-%m-%dT%H:%M:%SZ"), hoy.strftime("%Y-%m-%dT%H:%M:%SZ")

fecha_inicio, fecha_fin = obtener_fecha_30_dias_atras()

def obtener_datos_generacion():
    endpoint = 'generacion/estructura-generacion'
    url_base = "https://apidatos.ree.es/es/datos"
    url = f"{url_base}/{endpoint}"

    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    params = {
        "start_date": fecha_inicio,
        "end_date": fecha_fin,
        "time_trunc": "day"
    }

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:

        datos = response.json()

        print(f"Datos recibidos del endpoint {endpoint}:")
        
        datos_generacion = []

        for registro in datos.get("included", []):
            tipo = registro['attributes']['title']
            for valor in registro['attributes']['values']:
                fecha = valor['datetime']
                generacion = valor['value']
                datos_generacion.append({'Fecha': fecha, 'Generación': generacion, 'Tipo': tipo})

        df = pd.DataFrame(datos_generacion)
        
        df['Fecha'] = pd.to_datetime(df['Fecha'])

        return df
    else:
        print(f"Error al hacer la solicitud al endpoint {endpoint}: {response.status_code}")
        print(f"Respuesta de la API: {response.text}")
        return pd.DataFrame()

df_generacion = obtener_datos_generacion()

print("\nGeneración:")
print(df_generacion)


Datos recibidos del endpoint generacion/estructura-generacion:

Generación:
                       Fecha  Generación                    Tipo
0  2024-09-30 00:00:00+02:00   69976.900              Hidráulica
1  2024-10-01 00:00:00+02:00   41914.900              Hidráulica
2  2024-09-30 00:00:00+02:00    9405.600      Turbinación bombeo
3  2024-10-01 00:00:00+02:00    5895.200      Turbinación bombeo
4  2024-09-30 00:00:00+02:00  163999.500                 Nuclear
5  2024-10-01 00:00:00+02:00   97262.000                 Nuclear
6  2024-09-30 00:00:00+02:00    8456.700                  Carbón
7  2024-10-01 00:00:00+02:00    5079.100                  Carbón
8  2024-09-30 00:00:00+02:00    6690.019          Motores diésel
9  2024-10-01 00:00:00+02:00    3559.232          Motores diésel
10 2024-09-30 00:00:00+02:00    1308.322          Turbina de gas
11 2024-10-01 00:00:00+02:00     865.871          Turbina de gas
12 2024-09-30 00:00:00+02:00    2580.606        Turbina de vapor
13 2024-10-01 

In [14]:
import configparser
import mysql.connector
import requests
import pandas as pd
from datetime import datetime, timedelta

# Leer el archivo de configuración
config = configparser.ConfigParser()
config.read('config.ini')

# Obtener los valores del archivo de configuración
host = config['mysql']['host']
user = config['mysql']['user']
password = config['mysql']['password']

# Conectar a la base de datos
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password
)

# Función para obtener las fechas de 30 días atrás
def obtener_fecha_30_dias_atras():
    hoy = datetime.now()
    hace_30_dias = hoy - timedelta(days=30)  # Cambiado a 30 días
    return hace_30_dias.strftime("%Y-%m-%dT%H:%M:%SZ"), hoy.strftime("%Y-%m-%dT%H:%M:%SZ")

fecha_inicio, fecha_fin = obtener_fecha_30_dias_atras()

# Función para obtener datos de generación
def obtener_datos_generacion():
    endpoint = 'generacion/estructura-generacion'
    url_base = "https://apidatos.ree.es/es/datos"
    url = f"{url_base}/{endpoint}"

    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    params = {
        "start_date": fecha_inicio,
        "end_date": fecha_fin,
        "time_trunc": "day"
    }

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        datos = response.json()

        print(f"Datos recibidos del endpoint {endpoint}:")
        
        datos_generacion = []

        for registro in datos.get("included", []):
            tipo = registro['attributes']['title']
            for valor in registro['attributes']['values']:
                fecha = valor['datetime']
                generacion = valor['value']
                datos_generacion.append({'Fecha': fecha, 'Generación': generacion, 'Tipo': tipo})

        df = pd.DataFrame(datos_generacion)
        
        df['Fecha'] = pd.to_datetime(df['Fecha'])

        return df
    else:
        print(f"Error al hacer la solicitud al endpoint {endpoint}: {response.status_code}")
        print(f"Respuesta de la API: {response.text}")
        return pd.DataFrame()

# Obtener datos de generación
df_generacion = obtener_datos_generacion()

# Crear base de datos y tabla
cursor = conn.cursor()

# Eliminar la base de datos si existe
cursor.execute("DROP DATABASE IF EXISTS red_electrica")
cursor.execute("CREATE DATABASE red_electrica")

# Usar la nueva base de datos
cursor.execute("USE red_electrica")

# Crear tabla
cursor.execute("""
    CREATE TABLE generacion (
        id INT AUTO_INCREMENT PRIMARY KEY,
        fecha DATETIME NOT NULL,
        generacion FLOAT NOT NULL,
        tipo VARCHAR(50) NOT NULL
    )
""")

# Insertar datos en la tabla
for index, row in df_generacion.iterrows():
    cursor.execute("""
        INSERT INTO generacion (fecha, generacion, tipo) 
        VALUES (%s, %s, %s)
    """, (row['Fecha'], row['Generación'], row['Tipo']))

# Confirmar los cambios
conn.commit()

# Cerrar conexión
cursor.close()
conn.close()

print("Datos insertados en la base de datos 'red_electrica' con éxito.")


Datos recibidos del endpoint generacion/estructura-generacion:
Datos insertados en la base de datos 'red_electrica' con éxito.


In [15]:
import requests
import pandas as pd
import mysql.connector
import configparser

# Leer el archivo de configuración
config = configparser.ConfigParser()
config.read('config.ini')

# Obtener los valores del archivo de configuración
host = config['mysql']['host']
user = config['mysql']['user']
password = config['mysql']['password']
database = "red_electrica"

# Función para obtener datos de demanda
def demanda_datos(lang, category, widget, query):
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json"
    }

    endpoint = f"https://apidatos.ree.es/{lang}/datos/{category}/{widget}?{query}"
    response = requests.get(url=endpoint, headers=headers)

    if response.status_code != 200:
        print(f"Error al obtener datos: {response.status_code}")
        return pd.DataFrame()

    data = response.json()

    datetime_lista = []
    value_lista = []
    percentage_lista = []

    for value in data['included']:
        for content in value['attributes']['values']:
            porcentaje = content['percentage']
            valor = content['value']
            fecha = content['datetime']

            datetime_lista.append(fecha)
            value_lista.append(valor)
            percentage_lista.append(porcentaje)

    df = pd.DataFrame()
    df['datetime'] = datetime_lista
    df['percentage'] = percentage_lista
    df['value'] = value_lista

    df['datetime'] = df['datetime'].apply(lambda x: pd.to_datetime(x, errors='coerce').strftime('%d/%m/%Y') if pd.to_datetime(x, errors='coerce') is not pd.NaT else x)

    return df

# Obtener datos de demanda
lang = "es"
category = "demanda"
widget = "ire-general"
query = "start_date=2023-01-01T00:00&end_date=2023-01-31T23:59&time_trunc=day"
df_demand = demanda_datos(lang, category, widget, query)

# Conectar a la base de datos y almacenar datos
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

cursor = conn.cursor()

# Eliminar tabla si existe y crearla de nuevo
cursor.execute("DROP TABLE IF EXISTS demanda")
cursor.execute("""
    CREATE TABLE demanda (
        id INT AUTO_INCREMENT PRIMARY KEY,
        datetime DATE NOT NULL,
        percentage FLOAT NOT NULL,
        value FLOAT NOT NULL
    )
""")

# Insertar datos en la tabla
for index, row in df_demand.iterrows():
    cursor.execute("""
        INSERT INTO demanda (datetime, percentage, value) 
        VALUES (%s, %s, %s)
    """, (row['datetime'], row['percentage'], row['value']))

# Confirmar los cambios
conn.commit()

# Cerrar conexión
cursor.close()
conn.close()

print("Datos de demanda insertados en la base de datos 'red_electrica' con éxito.")


Error al obtener datos: 500
Datos de demanda insertados en la base de datos 'red_electrica' con éxito.
